# 경로 확인

In [70]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [71]:
%cd /content/drive/MyDrive/02.git/healthcare-hackathon
print('===')
!ls

/content/drive/MyDrive/02.git/healthcare-hackathon
===
backend  docker-compose.yml  frontend  langchain  model  README.md  temp.py


In [72]:
%cd model/fer2013-finetuning
!ls

/content/drive/MyDrive/02.git/healthcare-hackathon/model/fer2013-finetuning
configs			       MobileNet_v4_extra-train.ipynb  README.md
docs			       MobileNet_v4_init_train.ipynb   requirements.txt
MobileNet_model_structure.pth  models			       utils


# MobileNet 불러오기
- save로 모델 생성
- 이전에 학습했던 가중치 파일 적용

In [73]:
import sys
sys.path.append('/content/drive/MyDrive/02.git/healthcare-hackathon/model/fer2013-finetuning')

In [74]:
import torch
from models.classifier import create_emotion_classifier
from utils.dataset import create_dataloaders
from utils.trainer import EmotionTrainer
from configs.config import Config

# 모델 생성
model = create_emotion_classifier(
    num_classes=Config.NUM_CLASSES,
    backbone_name=Config.MODEL_NAME
)

torch.save(model, 'MobileNet_model_structure.pth')

In [75]:
!cp MobileNet_model_structure.pth ../fer2013-test/MobileNet_model_structure.pth

- 가중치 파일 구드 링크
- https://drive.google.com/file/d/1eGZ8Kb3jd_V9wYSI7BBgbxKrItcuvGg_/view?usp=drive_link

In [76]:
# 가중치 파일 다운로드(구글드라이브 링크)
file_id = "1eGZ8Kb3jd_V9wYSI7BBgbxKrItcuvGg_"

# gdown을 사용하여 파일 다운로드
!gdown {file_id} -O /content/MobileNet_finetuned.pth

Downloading...
From (original): https://drive.google.com/uc?id=1eGZ8Kb3jd_V9wYSI7BBgbxKrItcuvGg_
From (redirected): https://drive.google.com/uc?id=1eGZ8Kb3jd_V9wYSI7BBgbxKrItcuvGg_&confirm=t&uuid=91bf790a-de66-4fde-9e67-3d16f7050128
To: /content/MobileNet_finetuned.pth
100% 37.1M/37.1M [00:00<00:00, 77.7MB/s]


In [77]:
# 저장된 가중치 로드
checkpoint = torch.load('/content/MobileNet_finetuned.pth', map_location=Config.DEVICE)
model.load_state_dict(checkpoint)
print("✅ best_model_copy.pth 로드 완료")

# 모델의 디바이스 확인
model.to(Config.DEVICE)
device = next(model.parameters()).device
print(f"모델은 현재 {device}에 있습니다.")

✅ best_model_copy.pth 로드 완료
모델은 현재 cuda:0에 있습니다.


# 데이터셋 준비
- https://drive.google.com/file/d/1OOFyCEGTbFrsuYr1Q5Ds7QAvpiXkqyhT/view?usp=drive_link

In [78]:
file_id = "1OOFyCEGTbFrsuYr1Q5Ds7QAvpiXkqyhT"

# gdown을 사용하여 파일 다운로드
!gdown {file_id} -O /content/fer2013.zip

# 압축 해제
!unzip -q /content/fer2013.zip -d /content/fer2013

Downloading...
From (original): https://drive.google.com/uc?id=1OOFyCEGTbFrsuYr1Q5Ds7QAvpiXkqyhT
From (redirected): https://drive.google.com/uc?id=1OOFyCEGTbFrsuYr1Q5Ds7QAvpiXkqyhT&confirm=t&uuid=96237c40-f686-46b3-87c6-39b4dfee75c0
To: /content/fer2013.zip
100% 63.3M/63.3M [00:00<00:00, 161MB/s]
replace /content/fer2013/test/angry/PrivateTest_10131363.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: A


In [79]:
from sklearn.metrics import accuracy_score, classification_report
from tqdm import tqdm


# 테스트 데이터셋 로드
train_loader, val_loader = create_dataloaders(
    data_dir='/content/fer2013',
    batch_size=256,
    num_workers=2
)

# 모델을 평가 모드로 전환
model.eval()

# 예측 및 실제 레이블 저장
all_preds = []
all_labels = []

# 테스트 데이터셋에 대해 예측 수행
with torch.no_grad():
    for inputs, labels in tqdm(val_loader, desc="Evaluating"):  # tqdm으로 반복문 감싸기
        inputs, labels = inputs.to(Config.DEVICE), labels.to(Config.DEVICE)

        # 모델 예측
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)

        # 예측 및 실제 레이블 저장
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# 정확도 계산
accuracy = accuracy_score(all_labels, all_preds)
print(f"테스트 정확도: {accuracy * 100:.2f}%")

# 클래스 레이블 (예: 0부터 6까지의 7가지 클래스)
class_labels = ['angry', 'disgust', 'fear',
            'happy', 'neutral', 'sad', 'surprise']

# 분류 보고서 출력
report = classification_report(all_labels, all_preds, target_names=[f"Class {i}" for i in class_labels])
print(report)

Evaluating: 100%|██████████| 29/29 [00:20<00:00,  1.41it/s]

테스트 정확도: 62.50%
                precision    recall  f1-score   support

   Class angry       0.51      0.55      0.53       958
 Class disgust       0.63      0.35      0.45       111
    Class fear       0.47      0.33      0.39      1024
   Class happy       0.84      0.85      0.84      1774
 Class neutral       0.56      0.64      0.60      1233
     Class sad       0.49      0.53      0.51      1247
Class surprise       0.75      0.77      0.76       831

      accuracy                           0.62      7178
     macro avg       0.61      0.57      0.58      7178
  weighted avg       0.62      0.62      0.62      7178



In [80]:
!pip install -U -q transformers

In [81]:
from transformers import pipeline

# 사전 학습된 모델을 사용하여 이미지 분류 파이프라인 생성
pipe = pipeline("image-classification", model="trpakov/vit-face-expression")

from transformers import AutoImageProcessor, AutoModelForImageClassification

# 사전 학습된 모델에 맞는 이미지 전처리기 로드
processor = AutoImageProcessor.from_pretrained("trpakov/vit-face-expression", use_fast=True)

# 사전 학습된 이미지 분류 모델 로드
model_trpakov = AutoModelForImageClassification.from_pretrained("trpakov/vit-face-expression")

# 모델을 지정한 디바이스(CPU 또는 GPU)로 이동
model_trpakov.to(Config.DEVICE)

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Device set to use cuda:0


Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

ViTForImageClassification(
  (vit): ViTModel(
    (embeddings): ViTEmbeddings(
      (patch_embeddings): ViTPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViTEncoder(
      (layer): ModuleList(
        (0-11): 12 x ViTLayer(
          (attention): ViTAttention(
            (attention): ViTSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
            )
            (output): ViTSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ViTIntermediate(
            (dense): Linear(in_features=768, out_features=3072, bias=True)
            (intermed

In [82]:
# 테스트 데이터셋 로드(is_norm=False, [0,1] 로 정규화)
train_loader, val_loader = create_dataloaders(
    data_dir='/content/fer2013',
    batch_size=256,
    num_workers=2,
    is_norm=False
)

In [83]:
def huggingface_model_evaluation(model, processor, val_loader):
    # 모델을 평가 모드로 전환
    model.eval()

    # 예측 및 실제 레이블 저장
    all_preds = []
    all_labels = []

    # 테스트 데이터셋에 대해 예측 수행
    with torch.no_grad():
        for inputs, labels in tqdm(val_loader, desc="Evaluating"):
            # [0,1] 텐서를 PIL Image로 변환
            batch_images = []
            for img_tensor in inputs:
                # CHW → HWC 순서 변경 후 [0,1] → [0,255] 변환
                img_array = img_tensor.permute(1, 2, 0).numpy()
                img_array = (img_array * 255).astype(np.uint8)
                batch_images.append(Image.fromarray(img_array))

            # Hugging Face processor가 자동으로 모델에 맞는 전처리 수행
            processed_inputs = processor(images=batch_images, return_tensors="pt").to(Config.DEVICE)

            outputs = model(**processed_inputs)
            preds = torch.argmax(outputs.logits, dim=-1)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # 정확도 계산
    accuracy = accuracy_score(all_labels, all_preds)
    print(f"테스트 정확도: {accuracy * 100:.2f}%")

    # 클래스 레이블 (예: 0부터 6까지의 7가지 클래스)
    class_names = ['angry', 'disgust', 'fear',
                'happy', 'neutral', 'sad', 'surprise']

    # 분류 보고서 출력
    report = classification_report(all_labels, all_preds, target_names=[f"Class {i}" for i in class_names])
    print(report)

In [84]:
huggingface_model_evaluation(model_trpakov, processor, val_loader)

Evaluating: 100%|██████████| 29/29 [02:13<00:00,  4.61s/it]

테스트 정확도: 71.15%
                precision    recall  f1-score   support

   Class angry       0.63      0.65      0.64       958
 Class disgust       0.75      0.67      0.70       111
    Class fear       0.60      0.54      0.57      1024
   Class happy       0.90      0.88      0.89      1774
 Class neutral       0.68      0.67      0.68      1233
     Class sad       0.57      0.64      0.60      1247
Class surprise       0.83      0.81      0.82       831

      accuracy                           0.71      7178
     macro avg       0.71      0.69      0.70      7178
  weighted avg       0.72      0.71      0.71      7178



In [86]:
processor = AutoImageProcessor.from_pretrained("mo-thecreator/vit-Facial-Expression-Recognition")
model_mo = AutoModelForImageClassification.from_pretrained("mo-thecreator/vit-Facial-Expression-Recognition")
model_mo.to(Config.DEVICE)
huggingface_model_evaluation(model_mo, processor, val_loader)

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Evaluating: 100%|██████████| 29/29 [02:03<00:00,  4.27s/it]

테스트 정확도: 91.10%
                precision    recall  f1-score   support

   Class angry       0.91      0.87      0.89       958
 Class disgust       0.94      0.93      0.93       111
    Class fear       0.84      0.87      0.85      1024
   Class happy       0.97      0.96      0.96      1774
 Class neutral       0.88      0.94      0.91      1233
     Class sad       0.90      0.87      0.89      1247
Class surprise       0.95      0.92      0.93       831

      accuracy                           0.91      7178
     macro avg       0.91      0.91      0.91      7178
  weighted avg       0.91      0.91      0.91      7178



In [87]:
processor = AutoImageProcessor.from_pretrained("kriskrishna/vit-Facial-Expression-Recognition")
model_kriskrishna = AutoModelForImageClassification.from_pretrained("kriskrishna/vit-Facial-Expression-Recognition")
model_kriskrishna.to(Config.DEVICE)
huggingface_model_evaluation(model_kriskrishna, processor, val_loader)

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

preprocessor_config.json:   0%|          | 0.00/347 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/965 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/343M [00:00<?, ?B/s]

Evaluating: 100%|██████████| 29/29 [01:59<00:00,  4.11s/it]

테스트 정확도: 87.14%
                precision    recall  f1-score   support

   Class angry       0.83      0.84      0.83       958
 Class disgust       1.00      0.69      0.82       111
    Class fear       0.69      0.92      0.79      1024
   Class happy       0.95      0.96      0.95      1774
 Class neutral       0.96      0.75      0.84      1233
     Class sad       0.87      0.85      0.86      1247
Class surprise       0.94      0.92      0.93       831

      accuracy                           0.87      7178
     macro avg       0.89      0.85      0.86      7178
  weighted avg       0.88      0.87      0.87      7178

